In [1]:
# # Compute correlations ans variances
# correlations = pg.pairwise_corr(flights_df[numerical_columns])
# variances = flights_df[numerical_columns].var()

# # Select the feature with lower variance in each pair to drop
# to_drop = set()
# for feature1, feature2 in correlations[['X', 'Y']].values:
#     if variances[feature1] < variances[feature2]:
#         to_drop.add(feature1)
#     else:
#         to_drop.add(feature2)

# # Drop the low-variance correlated features
# flights_df = flights_df.drop(columns=to_drop)

# print(f"Features dropped based on variance: {to_drop}")

# flights_df.info()

In [2]:
import pandas as pd

import lightgbm as lgb
import xgboost as xgb

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score, f1_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [3]:
# Read csv
flights_df = pd.read_csv('../data/flight-delays/flights.csv')

# Show df
flights_df.info()

/var/folders/3f/d8gl3k3j0bn3wk_cqjvfm31w0000gn/T/ipykernel_20927/3213893161.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  flights_df = pd.read_csv('../data/flight-delays/flights.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 31 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [4]:
# Drop columns with more than 20% of values missing
flights_df = flights_df.loc[:, (flights_df.isnull().sum() / flights_df.shape[0] < 0.2).values]

# Show df
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5819079 entries, 0 to 5819078
Data columns (total 25 columns):
 #   Column               Dtype  
---  ------               -----  
 0   YEAR                 int64  
 1   MONTH                int64  
 2   DAY                  int64  
 3   DAY_OF_WEEK          int64  
 4   AIRLINE              object 
 5   FLIGHT_NUMBER        int64  
 6   TAIL_NUMBER          object 
 7   ORIGIN_AIRPORT       object 
 8   DESTINATION_AIRPORT  object 
 9   SCHEDULED_DEPARTURE  int64  
 10  DEPARTURE_TIME       float64
 11  DEPARTURE_DELAY      float64
 12  TAXI_OUT             float64
 13  WHEELS_OFF           float64
 14  SCHEDULED_TIME       float64
 15  ELAPSED_TIME         float64
 16  AIR_TIME             float64
 17  DISTANCE             int64  
 18  WHEELS_ON            float64
 19  TAXI_IN              float64
 20  SCHEDULED_ARRIVAL    int64  
 21  ARRIVAL_TIME         float64
 22  ARRIVAL_DELAY        float64
 23  DIVERTED             int64  
 24

In [5]:
# Filter out rows where ORIGIN_AIRPORT or DESTINATION_AIRPORT contains numbers
flights_df = flights_df[~flights_df['ORIGIN_AIRPORT'].str.isnumeric().astype(bool)]
flights_df = flights_df[~flights_df['DESTINATION_AIRPORT'].str.isnumeric().astype(bool)]

# Columns to convert to categorical
categorical_columns = [
    'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK',
    'AIRLINE', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
    'DIVERTED', 'CANCELLED'
]

# Convert specified columns to categorical
flights_df[categorical_columns] = flights_df[categorical_columns].astype('category')

# Convert the rest of the columns to numerical
# Identify numerical columns by excluding categorical columns
numerical_columns = flights_df.columns.difference(categorical_columns).to_list()

# Convert remaining columns to numerical (float for precision)
flights_df[numerical_columns] = flights_df[numerical_columns].apply(pd.to_numeric, errors='coerce')

# Drop cancelled flights
flights_df = flights_df[flights_df['CANCELLED'] == 0]
flights_df = flights_df.drop(columns=['CANCELLED'])
categorical_columns.remove('CANCELLED')

# Create target column and drop delayed flights
flights_df['DELAYED'] = flights_df['ARRIVAL_DELAY'].apply(lambda arrival_delay: arrival_delay >=15)
flights_df = flights_df.drop(columns=['ARRIVAL_DELAY'])
numerical_columns.remove('ARRIVAL_DELAY')

# Show df
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5245484 entries, 0 to 5819078
Data columns (total 24 columns):
 #   Column               Dtype   
---  ------               -----   
 0   YEAR                 category
 1   MONTH                category
 2   DAY                  category
 3   DAY_OF_WEEK          category
 4   AIRLINE              category
 5   FLIGHT_NUMBER        int64   
 6   TAIL_NUMBER          category
 7   ORIGIN_AIRPORT       category
 8   DESTINATION_AIRPORT  category
 9   SCHEDULED_DEPARTURE  int64   
 10  DEPARTURE_TIME       float64 
 11  DEPARTURE_DELAY      float64 
 12  TAXI_OUT             float64 
 13  WHEELS_OFF           float64 
 14  SCHEDULED_TIME       float64 
 15  ELAPSED_TIME         float64 
 16  AIR_TIME             float64 
 17  DISTANCE             int64   
 18  WHEELS_ON            float64 
 19  TAXI_IN              float64 
 20  SCHEDULED_ARRIVAL    int64   
 21  ARRIVAL_TIME         float64 
 22  DIVERTED             category
 23  DELAYED     

In [6]:
# # Preprocessing for numerical data
# numerical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
#     # ('scaler', StandardScaler())                    # Standardize numerical features
# ])

# # Preprocessing for categorical data
# categorical_transformer = Pipeline(steps=[
#     ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with mode
#     ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode categorical features
# ])

# preprocessor = ColumnTransformer(
#     [
#         ('num', numerical_transformer, numerical_columns),
#         ('cat', categorical_transformer, categorical_columns)
#     ]
# )

# # Assign sparse array
# X = preprocessor.fit_transform(flights_df.drop(columns=['DELAYED']))
# y = flights_df['DELAYED'].values

# # Create train and test split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size = 0.3 
# )

In [7]:
# print("XGBoost")

# clf = xgb.XGBClassifier()
# clf.fit(X_train, y_train)

# y_pred = clf.predict(X_test)

# roc_auc = roc_auc_score(y_test, y_pred)
# print(f"ROC AUC: {roc_auc}")

# recall = recall_score(y_test, y_pred)
# print(f"Recall: {recall}")

# f1 = f1_score(y_test, y_pred)
# print(f"F1: {f1}")

In [8]:
# Set Pandas output
set_config(transform_output = "pandas")
# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with median
    # ('scaler', StandardScaler())                   # Standardize numerical features
])
flights_df[numerical_columns] = numerical_transformer.fit_transform(flights_df[numerical_columns])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),                       # Impute missing values with mode
    # ('onehot', OneHotEncoder(handle_unknown='ignore'))     # One-hot encode categorical features
])
flights_df[categorical_columns] = categorical_transformer.fit_transform(flights_df[categorical_columns])

# Convert specified columns to categorical, again
flights_df[categorical_columns] = flights_df[categorical_columns].astype('category')

# Assign from data frames
X = flights_df.drop(columns=['DELAYED'])
y = flights_df['DELAYED']

# Create train and test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.3 
)

# Show df
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5245484 entries, 0 to 5819078
Data columns (total 23 columns):
 #   Column               Dtype   
---  ------               -----   
 0   YEAR                 category
 1   MONTH                category
 2   DAY                  category
 3   DAY_OF_WEEK          category
 4   AIRLINE              category
 5   FLIGHT_NUMBER        float64 
 6   TAIL_NUMBER          category
 7   ORIGIN_AIRPORT       category
 8   DESTINATION_AIRPORT  category
 9   SCHEDULED_DEPARTURE  float64 
 10  DEPARTURE_TIME       float64 
 11  DEPARTURE_DELAY      float64 
 12  TAXI_OUT             float64 
 13  WHEELS_OFF           float64 
 14  SCHEDULED_TIME       float64 
 15  ELAPSED_TIME         float64 
 16  AIR_TIME             float64 
 17  DISTANCE             float64 
 18  WHEELS_ON            float64 
 19  TAXI_IN              float64 
 20  SCHEDULED_ARRIVAL    float64 
 21  ARRIVAL_TIME         float64 
 22  DIVERTED             category
dtypes: category(

In [9]:
clf = xgb.XGBClassifier(enable_categorical=True)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC: {roc_auc}")

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

f1 = f1_score(y_test, y_pred)
print(f"F1: {f1}")

ROC AUC: 0.9768860321313875
Recall: 0.9589685882817661
F1: 0.9682069156200515
